<a href="https://colab.research.google.com/github/NehalH/RupeeClassifier_MobileNet/blob/main/RupeeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define an ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Specify the path to your dataset on Google Drive
dataset_path = '/content/drive/MyDrive/A-eye/Datasets/Indian Currency Dataset'

# Manually specify the classes
class_names = ['10', '20', '50', '100', '200']

# Create a data generator for training images with manual class specification
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  # Manually specify the class names
)

Found 1269 images belonging to 5 classes.


In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNet base model (pre-trained on ImageNet)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build your custom model on top of MobileNet
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(train_generator, epochs=10)  # Adjust the number of epochs based on your requirements

# Save the model in .tf format
tf_model_path = '/content/drive/MyDrive/A-eye/Models/Rupee_Classifier/rupee_classifier.tf'
model.save(tf_model_path, save_format='tf')

17225924/17225924 [==============================] - 0s 0us/step
Epoch 1/10
40/40 [==============================] - 540s 13s/step - loss: 0.3846 - accuracy: 0.8629
Epoch 2/10
40/40 [==============================] - 198s 5s/step - loss: 0.1620 - accuracy: 0.9598
Epoch 3/10
40/40 [==============================] - 201s 5s/step - loss: 0.0962 - accuracy: 0.9740
Epoch 4/10
40/40 [==============================] - 201s 5s/step - loss: 0.0883 - accuracy: 0.9701
Epoch 5/10
40/40 [==============================] - 202s 5s/step - loss: 0.0531 - accuracy: 0.9779
Epoch 6/10
40/40 [==============================] - 207s 5s/step - loss: 0.0502 - accuracy: 0.9842
Epoch 7/10
40/40 [==============================] - 206s 5s/step - loss: 0.0886 - accuracy: 0.9685
Epoch 8/10
40/40 [==============================] - 219s 5s/step - loss: 0.1102 - accuracy: 0.9732
Epoch 9/10
40/40 [==============================] - 208s 5s/step - loss: 0.0419 - accuracy: 0.9850
Epoch 10/10
40/40 [========================

In [ ]:
import tensorflow as tf

# Convert the model to Unquantized TensorFlow Lite
unquantized_converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
unquantized_tflite_model = unquantized_converter.convert()

# Save the TFLite model
with open('/content/drive/MyDrive/A-eye/Models/Rupee_Classifier/rupee_classifier_unquant.tflite', 'wb') as f:
    f.write(unquantized_tflite_model)


In [ ]:
# Convert the model to Quantized TensorFlow Lite

quantized_converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)

# Enable quantization
quantized_converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = quantized_converter.convert()

# Save the Quantized TFLite model
with open('/content/drive/MyDrive/A-eye/Models/Rupee_Classifier/rupee_classifier_quant.tflite', 'wb') as f:
    f.write(quantized_tflite_model)

In [ ]:
# Path to save the labels file
labels_path = '/content/drive/MyDrive/A-eye/Models/Rupee_Classifier/labels.txt'

# Write class names to the labels file
with open(labels_path, 'w') as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")

##Testing

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np

# Load the Quantized TFLite model
tflite_model_path = '/content/drive/MyDrive/A-eye/Models/Rupee_Classifier/rupee_classifier_quant.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Load the labels
labels_path = '/content/drive/MyDrive/A-eye/Models/Rupee_Classifier/labels.txt'
with open(labels_path, 'r') as f:
    labels = [line.strip() for line in f.readlines()]

# Load a custom image
custom_image_path = '/content/test_dataset/8etrnzyzuz1b1.jpg'  # Replace with the path to your image
custom_image = Image.open(custom_image_path).resize((224, 224))

# Preprocess the image
input_tensor_index = interpreter.get_input_details()[0]['index']
input_tensor = np.expand_dims(custom_image, axis=0) / 255.0  # Assuming normalization to [0, 1]
interpreter.set_tensor(input_tensor_index, input_tensor.astype(np.float32))

# Run inference
interpreter.invoke()

# Get the output
output_tensor_index = interpreter.get_output_details()[0]['index']
output_tensor = interpreter.get_tensor(output_tensor_index)

# Get the predicted class index
predicted_class_index = np.argmax(output_tensor)

# Get the predicted class label
predicted_class_label = labels[predicted_class_index]

print(f"Predicted Class: {predicted_class_label}")


Predicted Class: 20
